In [ ]:
from neuprint import Client
from neuprint import fetch_neurons
from neuprint import NeuronCriteria as NC
from neuprint import SynapseCriteria as SC


with open('auth_token.txt', 'r') as f:
    token = f.read()

c = Client('neuprint.janelia.org', dataset='manc:v1.0', token=token)
c.fetch_version()

import pandas as pd

from matplotlib import rcParams

rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 9


xl = pd.read_excel('Putative IDs.xlsx')
xl.columns = xl.loc[0,:]
xl.drop(0, inplace=True)
# drop first column
xl.drop(xl.columns[0], axis=1, inplace=True)

putative_cdn =  xl['CDN'].loc[~xl['CDN'].isnull()].values.astype(int)

pcdn = fetch_neurons(
    NC(
        bodyId=putative_cdn,
    ),
    client=c
)[0]

In [ ]:
import bokeh.palettes
from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_notebook
import numpy as np
from neuprint import fetch_synapses
from neuprint import SynapseCriteria as SC
output_notebook()

pcdn_gab = pcdn[pcdn['ntGabaProb'] >= 0.7]

from neuprint import fetch_adjacencies

# _, pcdn_conn = fetch_adjacencies(
#     targets = NC(
#         bodyId=pcdn_gab['bodyId'].values,
#     ),
#     min_total_weight = 5,
# )

_, pcdn_conn = fetch_adjacencies(
    targets = NC(
        bodyId=pcdn['bodyId'].values,
    ),
    min_total_weight = 5,
)

_, pcdn_conn_out = fetch_adjacencies(
    sources = NC(
        bodyId=pcdn_gab['bodyId'].values,
    ),
    min_total_weight = 5,
)

# _, pcdn_conn_out = fetch_adjacencies(
#     sources = NC(
#         bodyId=pcdn['bodyId'].values,
#     ),
#     min_total_weight = 5,
# )


import seaborn as sns
import matplotlib.pyplot as plt

matrix = pcdn_conn_out.pivot_table(index='bodyId_pre', columns='bodyId_post', values='weight', aggfunc='sum', fill_value=0)
post_mat = pcdn_conn.pivot_table(index='bodyId_post', columns='bodyId_pre', values='weight', aggfunc='sum', fill_value=0)


def cosine_similarity(matrix):
    """
    Calculate the cosine similarity for a matrix.
    """
    from sklearn.metrics.pairwise import cosine_similarity

    # Calculate cosine similarity between rows
    cosine_sim_matrix = cosine_similarity(matrix)

    # Create a DataFrame with cosine similarity matrix
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=matrix.index, columns=matrix.index)

    return cosine_sim_df


# Calculate cosine similarity between rows
cosine_sim_matrix = cosine_similarity(matrix)

# Create a DataFrame with cosine similarity matrix
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=matrix.index, columns=matrix.index)

cosine_sim_post = cosine_similarity(post_mat)
cosine_sim_post_df = pd.DataFrame(cosine_sim_post, index=post_mat.index, columns=post_mat.index)

# Order the sorted matrix with hierarchical clustering
from scipy.cluster.hierarchy import linkage, dendrogram

# Calculate the linkage matrix
linkage_matrix = linkage(cosine_sim_post_df, 'average')

# Plot the dendrogram
plt.figure(figsize=(10,10))
r = dendrogram(linkage_matrix, labels=cosine_sim_post_df.index, orientation='left', leaf_font_size=10, no_plot=True)

sns.heatmap(
    cosine_sim_post_df.loc[r['ivl']][r['ivl']],
    #cosine_sim_df.loc[r['ivl']][r['ivl']],
    cmap='Blues',
    square=True,
    xticklabels=False,
    yticklabels=False,
    cbar_kws={'label': 'Cosine similarity of output vectors'},
    vmin=0,
    vmax=1,
)

plt.gcf().set_size_inches(10,10)

plt.gcf().savefig('All CDN morphology inputs cosine similarity.svg')

In [ ]:
sns.heatmap(
    #post_mat.loc[r['ivl']],
    matrix.loc[r['ivl']],
    #cosine_sim_df.loc[sorted_rows][sorted_rows] * cosine_sim_post_df.loc[sorted_rows][sorted_rows],
    cmap='Reds',
    square=False,
    xticklabels=False,
    yticklabels=False,
    cbar_kws={'label': 'Synapse count'},
    vmin=0,
    vmax = 50,
)

plt.gca().set_xlabel('')
plt.gca().set_ylabel('')
plt.gcf().set_size_inches(20,5)

plt.gcf().savefig('CDN output synapse count.png')

In [ ]:
cluster1 = [15860, 17405, 23216, 16787, 17301]
cluster2 = [14640, 34562, 16090, 18039, 101455, 16410, 101860]
cluster2_with_nongaba = [11394, 14097,22477, 155389, 16647, 16781, 14640, 34562, 16090, 18039, 101455, 16410, 101860]
cluster3 = [21196, 21732, 14457, 18011, 15861, 16676]
cluster4 = [13856, 14879, 14593, 16934, 14933, 15461]
cluster5 = [17452, 23036, 19902, 14371, 16252, 13174, 13605]

clusters = [cluster1, cluster2, cluster3, cluster4, cluster5]

In [ ]:
def plot_reciprocity(
    cluster : list[int],
    ax = None,
    **plot_kwargs,
):

    _, cluster_out_conns = fetch_adjacencies(
        sources = NC(
            bodyId = cluster,
        )
    )

    _, cluster_in_conns = fetch_adjacencies(
        sources = NC(
            bodyId = cluster_out_conns['bodyId_post'].unique(),
        ),
        targets = NC(
            bodyId = cluster,
        )
    )

    cluster_out_conns

    merged_cluster = cluster_in_conns.merge(
        cluster_out_conns,
        left_on='bodyId_pre',
        right_on='bodyId_post',
        suffixes=('_in', '_out')
    )

    merged_cluster = merged_cluster.groupby(['bodyId_post_out']).agg({'weight_out': 'sum', 'weight_in': 'sum'}).reset_index()
    #print(merged_cluster)
    if ax is None:
        fig, ax = plt.subplots()

    plot_kwargs = {
        'alpha': 0.5,
        'marker' : 'o',
        'linewidth' : 0,
        **plot_kwargs,
    }

    ax.scatter(
        merged_cluster['weight_in'],
        merged_cluster['weight_out'],
        **plot_kwargs,
    )

    ax.set_xlabel('Input synapse count')
    ax.set_ylabel('Output synapse count')

    # compute spearman correlation
    from scipy.stats import spearmanr
    rho, p = spearmanr(merged_cluster['weight_in'], merged_cluster['weight_out'])
    #ax.set_title(f'Spearman rho: {rho:.2f}, p={p:.2e}')
    print(f'Spearman rho: {rho:.2f}, p={p:.2e}')
    return ax

fig, axs = plt.subplots(1,5, figsize=(15,3))

for cluster, ax in zip(clusters, axs.flatten()):
    plot_reciprocity(cluster, ax=ax, color = '#000000')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    #ax.set_aspect('equal')
                     


plt.tight_layout()
#fig.savefig('CDN cluster reciprocity clusterwide.png')

# cellwise_reciprocity = 

# sns.heatmap(
#     merged_cluster['weight_in']/merged_cluster['weight_out'],
#     cmap='Reds',
#     square=False,
#     xticklabels=False,
#     yticklabels=False,
#     cbar_kws={'label': 'Synapse count'},
#     vmin=0,
#     vmax = 50,
# )

In [ ]:
CINNABAR = '#db544b'
LAPIS = '#2d66a5'
BYZANTINE = '#D342BE'
SHAMROCK = '#33A358'

roi_mesh = c.fetch_roi_mesh('ANm')

from bokeh.plotting import figure, show, output_notebook
output_notebook()
def plot_neurons(
        source_df, idx : int,
        flattened_ax : int = 1,
        on_p = None,
        weight : float = 1,
        color : str = '#000000',
        plot_synapses : bool = False,
        roi_mesh = c.fetch_roi_mesh('ANm'),
        ):

        skelee = c.fetch_skeleton(source_df.iloc[idx]['bodyId'])

        skelee.radius = skelee.radius.astype(float)/(20)
        # Join parent/child nodes for plotting as line segments below.
        # (Using each row's 'link' (parent) ID, find the row with matching rowId.)
        skelee = skelee.merge(skelee, 'inner',
                                left_on=['link'],
                                right_on=['rowId'],
                                suffixes=['_child', '_parent'])
        if on_p is None:
            p = figure(
                match_aspect = True
            )
            p.y_range.flipped = False
        else:
            p = on_p

        p.toolbar.logo = None
        p.toolbar_location = None

        axes = [x for x in range(3) if x != flattened_ax]
        if not roi_mesh is None:
            if not isinstance(roi_mesh, list):
                verts = np.array([
                    [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
                    for row in str(roi_mesh).split('\\n') if row.startswith('v')
                ])
            else:
                verts = np.array([
                    [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
                    for row in str(roi_mesh[0]).split('\\n') if row.startswith('v')
                ])
                for mesh in roi_mesh:
                    verts = np.concatenate((
                        verts,
                        np.array([
                            [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
                            for row in str(mesh).split('\\n') if row.startswith('v')
                        ])
                    ))

            speckle_size = 1.0
            p.scatter(verts[:,0],verts[:,1],size=speckle_size,line_alpha=0.0, fill_color = (0,0,0), fill_alpha = 0.5)

        str_labels = ['x', 'y', 'z']

        as_str = [str_labels[i] for i in axes]

        # Plot skeleton segments (in 2D)
        p.segment(
            x0=f'{as_str[0]}_child', x1=f'{as_str[0]}_parent',
            y0=f'{as_str[1]}_child', y1=f'{as_str[1]}_parent',
            line_width = 'radius_parent',
            alpha = 0.7*weight,
            color = color,
            source=skelee
        )

        if plot_synapses:
            syns = fetch_synapses(NC(bodyId=source_df.iloc[idx]['bodyId']))
            pre = syns.loc[syns['type'] == 'pre']
            p.scatter(
                pre[as_str[0]], pre[as_str[1]],
                size = 3, fill_color=BYZANTINE,
                fill_alpha = 0.9, line_color = None,
                line_alpha = 0.0, line_width=0,
            )
            
            post = syns.loc[syns['type'] == 'post']
            
            p.scatter(
                post[as_str[0]], post[as_str[1]],
                size = 3, fill_color=SHAMROCK,
                fill_alpha = 0.9, line_color=None,
                line_width = 0,
            )        

        p.renderers.insert(0,p.renderers.pop())

        if not (roi_mesh is None or isinstance(roi_mesh, list)):
            lims = {
                'x' : Range1d(18000,30000),
                'y' : Range1d(0,20000),
                'z' : Range1d(0,20000)
            }

            if as_str[0] in lims:
                p.x_range = lims[as_str[0]]
            if as_str[1] in lims:
                p.y_range = lims[as_str[1]]

        #p.y_range = Range1d(0,20000)
        #p.x_range = Range1d(18000,30000)
        p.xgrid.visible = False
        p.ygrid.visible = False
        p.axis.visible = False
        return p

from bokeh.plotting import gridplot

from bokeh.io import export_png

In [ ]:

cdn_df = fetch_neurons(
    NC(
        bodyId = cluster2
    ),
    client=c
)[0]

# color palette
pal = bokeh.palettes.Set1_7

# make a grid
on_ps = [
     plot_neurons(cdn_df, 0, 0, color = pal[0]),
     plot_neurons(cdn_df, 0, 1, color = pal[0]),
     plot_neurons(cdn_df, 0, 2, color = pal[0]),
]
for idx in range(1, len(cdn_df)):
     plot_neurons(cdn_df, idx, 0, on_p = on_ps[0], color = pal[idx%len(pal)]) 
     plot_neurons(cdn_df, idx, 1, on_p = on_ps[1], color = pal[idx%len(pal)])
     plot_neurons(cdn_df, idx, 2, on_p = on_ps[2], color = pal[idx%len(pal)])

grid_time = gridplot([on_ps])

show(grid_time)
#export_png(grid_time, filename = 'CDN_cluster2_together_cells.png')


In [ ]:
_, upstream_c2 = fetch_adjacencies(
    targets = NC(
        bodyId = cluster2
    ),
)

upstream_c2_grouped = upstream_c2.groupby(
    'bodyId_pre'
).agg(
    {
        'weight' : 'sum'
    }
).sort_values(
    by='weight',
    ascending=False
).reset_index().set_index('bodyId_pre')

upstream_c2_grouped['numerical_idx'] = range(len(upstream_c2_grouped))


cdn_in, _ = fetch_neurons(
    NC(
        bodyId = upstream_c2_grouped.index.values
    )
)

cdn_in.set_index('bodyId', inplace=True)

_, downstream_c2 = fetch_adjacencies(
    sources = NC(
        bodyId = cluster2
    ),
)

downstream_c2_grouped = downstream_c2.groupby(
    'bodyId_post'
).agg(
    {
        'weight' : 'sum'
    }
).sort_values(
    by='weight',
    ascending=False
).reset_index().set_index('bodyId_post')

downstream_c2_grouped['numerical_idx'] = range(len(downstream_c2_grouped))

cdn_out, _ = fetch_neurons(
    NC(
        bodyId = downstream_c2_grouped.index.values
    )
)

cdn_out.set_index('bodyId', inplace=True)


In [ ]:
downstream_c2_grouped['output_type'] = 'unknown'

downstream_c2_grouped['output_type'][
    cdn_out[cdn_out['outputRois'].apply(lambda x: len(x) >= 2)].index.values
] = 'interneuropil'

downstream_c2_grouped['output_type'][
     cdn_out[cdn_out['pre']/cdn_out['post'] < 1e-2].index.values
] = 'descending'

downstream_c2_grouped['output_type'][
    cdn_out[
        cdn_out['outputRois'].apply(lambda x: x == ['ANm']) *
        cdn_out['inputRois'].apply(lambda x: x == ['ANm'])
    ].index.values
] = 'interneuron'

In [ ]:
downstream_c2_normed = downstream_c2_grouped.copy()
downstream_c2_normed['weight'] = downstream_c2_normed['weight']/downstream_c2_normed['weight'].max()

downstream_c2_normed = downstream_c2_normed[downstream_c2_normed['weight'] > 0.01]

downstream_c2_normed.rename(columns={'bodyId_post' : 'bodyId'}, inplace=True)

In [ ]:
all_downstream_synapses_f, all_downstream_synapses_x = plt.subplots(nrows = 1, ncols = 1, figsize=(9,1.5))

all_downstream_synapses_x.plot(
    downstream_c2_grouped[downstream_c2_grouped['output_type'] == 'unknown']['numerical_idx'].values,
    downstream_c2_grouped[downstream_c2_grouped['output_type'] == 'unknown']['weight'].values,
    'ok'
)

all_downstream_synapses_x.plot(
    downstream_c2_grouped[downstream_c2_grouped['output_type'] == 'interneuropil']['numerical_idx'],
    downstream_c2_grouped[downstream_c2_grouped['output_type'] == 'interneuropil']['weight'].values,
    color = SHAMROCK,
    marker = 'o',
    linewidth = 0,
    alpha = 0.9,
)

all_downstream_synapses_x.plot(
    downstream_c2_grouped[downstream_c2_grouped['output_type'] == 'descending']['numerical_idx'],
    downstream_c2_grouped[downstream_c2_grouped['output_type'] == 'descending']['weight'].values,
    color = LAPIS,
    marker = 'o',
    linewidth = 0,
    alpha = 0.9
)

all_downstream_synapses_x.plot(
    downstream_c2_grouped[downstream_c2_grouped['output_type'] == 'interneuron']['numerical_idx'],
    downstream_c2_grouped[downstream_c2_grouped['output_type'] == 'interneuron']['weight'].values,
    color = '#58479E',
    marker = 'o',
    linewidth = 0,
    alpha = 0.9
)

all_downstream_synapses_x.set_ylabel('Synapse count')
all_downstream_synapses_x.set_xlabel('Postsynaptic cells')
#all_downstream_synapses_x.set_xticks([])
all_downstream_synapses_x.spines['right'].set_visible(False)
all_downstream_synapses_x.spines['top'].set_visible(False)
all_downstream_synapses_x.spines['bottom'].set_visible(False)

all_downstream_synapses_f.savefig('CDN_cluster2_downstream_synapse_count_by_category.svg')

all_downstream_synapses_x.set_xlim(-1, 100)
all_downstream_synapses_f.savefig('CDN_cluster2_downstream_synapse_count_by_category_zoom.svg')


In [ ]:
# Plot all interneuropil neurons
internp = cdn_out[downstream_c2_grouped['output_type'] == 'interneuropil']
internp['weight'] = downstream_c2_grouped.loc[internp.index.values]['weight']
internp['bodyId'] = internp.index.values
internp.sort_values(by='weight', ascending=False, inplace=True)

max_val = downstream_c2_grouped['weight'].max()

# all_rois = [c.fetch_roi_mesh(roiname) for roiname in ['ANm', 'LegNp(T1)(L)',
#  'LegNp(T1)(R)',
#  'LegNp(T2)(L)',
#  'LegNp(T2)(R)',
#  'LegNp(T3)(L)',
#  'LegNp(T3)(R)',]
# ]

# # make a grid
# on_ps = [
#      plot_neurons(internp, 0, 0, color = SHAMROCK, weight = internp['weight'].iloc[0]/max_val, roi_mesh=all_rois),
#      plot_neurons(internp, 0, 1, color = SHAMROCK, weight = internp['weight'].iloc[0]/max_val, roi_mesh=all_rois),
#      plot_neurons(internp, 0, 2, color = SHAMROCK, weight = internp['weight'].iloc[0]/max_val, roi_mesh=all_rois),
# ]
# for idx in range(1, 51):
#      plot_neurons(internp, idx, 0, on_p = on_ps[0], color = SHAMROCK, weight = internp['weight'].iloc[idx]/max_val, roi_mesh=None) 
#      plot_neurons(internp, idx, 1, on_p = on_ps[1], color = SHAMROCK, weight = internp['weight'].iloc[idx]/max_val, roi_mesh = None)
#      plot_neurons(internp, idx, 2, on_p = on_ps[2], color = SHAMROCK, weight = internp['weight'].iloc[idx]/max_val, roi_mesh = None)

# grid_time = gridplot([on_ps])

# #show(grid_time)

# show(grid_time)

# export_png(grid_time, filename = 'CDN_cluster2_interneuropil.png')
# #e




In [ ]:
internp['weight'].sum()/downstream_c2_grouped['weight'].sum()

In [ ]:
# Plot all interneuron downstream neurons
intern = cdn_out[downstream_c2_grouped['output_type'] == 'interneuron']
intern['weight'] = downstream_c2_grouped.loc[intern.index.values]['weight']
intern['bodyId'] = intern.index.values
intern.sort_values(by='weight', ascending=False, inplace=True)

max_val = downstream_c2_grouped['weight'].max()

# # make a grid
# on_ps = [
#      plot_neurons(intern, 0, 0, color = '#58479E', weight = intern['weight'].iloc[0]/max_val),
#      plot_neurons(intern, 0, 1, color = '#58479E', weight = intern['weight'].iloc[0]/max_val),
#      plot_neurons(intern, 0, 2, color = '#58479E', weight = intern['weight'].iloc[0]/max_val),
# ]
# for idx in range(1, 51):
#      plot_neurons(intern, idx, 0, on_p = on_ps[0], color = '#58479E', weight = intern['weight'].iloc[idx]/max_val, roi_mesh=None) 
#      plot_neurons(intern, idx, 1, on_p = on_ps[1], color = '#58479E', weight = intern['weight'].iloc[idx]/max_val, roi_mesh = None)
#      plot_neurons(intern, idx, 2, on_p = on_ps[2], color = '#58479E', weight = intern['weight'].iloc[idx]/max_val, roi_mesh = None)

# grid_time = gridplot([on_ps])

# #show(grid_time)

# show(grid_time)

# export_png(grid_time, filename = 'CDN_cluster2_interneurons.png')


In [ ]:
intern['weight'].sum()/downstream_c2_grouped['weight'].sum()

In [ ]:
# Plot all interneuron downstream neurons
descn = cdn_out[downstream_c2_grouped['output_type'] == 'descending']
descn['weight'] = downstream_c2_grouped.loc[descn.index.values]['weight']
descn['bodyId'] = descn.index.values
descn.sort_values(by='weight', ascending=False, inplace=True)

max_val = downstream_c2_grouped['weight'].max()

# make a grid
on_ps = [
     plot_neurons(descn, 0, 0, color = LAPIS, weight = descn['weight'].iloc[0]/max_val),
     plot_neurons(descn, 0, 1, color = LAPIS, weight = descn['weight'].iloc[0]/max_val),
     plot_neurons(descn, 0, 2, color = LAPIS, weight = descn['weight'].iloc[0]/max_val),
]
for idx in range(1, 51):
     try:
          plot_neurons(descn, idx, 0, on_p = on_ps[0], color = LAPIS, weight = descn['weight'].iloc[idx]/max_val, roi_mesh = None) 
          plot_neurons(descn, idx, 1, on_p = on_ps[1], color = LAPIS, weight = descn['weight'].iloc[idx]/max_val, roi_mesh = None)
          plot_neurons(descn, idx, 2, on_p = on_ps[2], color = LAPIS, weight = descn['weight'].iloc[idx]/max_val, roi_mesh = None)
     except:
          pass

grid_time = gridplot([on_ps])

#show(grid_time)

show(grid_time)

export_png(grid_time, filename = 'CDN_cluster2_motorneurons.png')


In [ ]:
descn['weight'].sum()/downstream_c2_grouped['weight'].sum()

In [ ]:
_, internp_by_roi = fetch_neurons(
    NC(bodyId=internp[internp['weight']>20]['bodyId'].values)
)

roiwise_internp = internp_by_roi.pivot_table(index='roi', columns='bodyId', values='pre', aggfunc='sum', fill_value=0)
roiwise_internp = roiwise_internp.reindex(internp[internp['weight']>20]['bodyId'].values, axis = 'columns')
roiwise_internp = roiwise_internp.loc[['ANm', 'LegNp(T1)(L)','LegNp(T1)(R)',
 'LegNp(T2)(L)',
 'LegNp(T2)(R)',
 'LegNp(T3)(L)',
 'LegNp(T3)(R)',]]
gs = plt.GridSpec(5, 1)

cdn_weights_x = plt.subplot(gs[0, :])

cdn_weights_x.plot(internp.loc[roiwise_internp.columns.values]['weight'].values, color = 'k')
cdn_weights_x.spines['right'].set_visible(False)
cdn_weights_x.spines['top'].set_visible(False)
cdn_weights_x.spines['bottom'].set_visible(False)
cdn_weights_x.set_xlim(-0.5, roiwise_internp.shape[1])
cdn_weights_x.set_xticks([])
cdn_weights_x.set_xlabel('')
cdn_weights_x.set_ylabel('Synapses\nfrom CDNs')

heatmap_ax = plt.subplot(gs[1:, :])

sns.heatmap(
    roiwise_internp,
    cmap='Greens',
    square=False,
    xticklabels=False,
    yticklabels=True,
    cbar = True,
    #cbar_kws={'label': 'Synapse count'},
    vmin=0,
    vmax = 500,
    ax = heatmap_ax
)

plt.gcf().set_size_inches(5,3)

plt.gcf().savefig('CDN_cluster2_out_interneuropil_heatmap_with_cbar.svg')

In [ ]:
# Scatter plot of intern

_, intern_from_cdn_conn = fetch_adjacencies(
    sources = NC(
        bodyId = cluster2
    ),
    targets = NC(
        bodyId = intern['bodyId'].values
    )
)

_, intern_to_cdn_conn = fetch_adjacencies(
    sources = NC(
        bodyId = intern['bodyId'].values
    ),
    targets = NC(
        bodyId = cluster2
    )
)

mutual_conn = intern_from_cdn_conn.merge(
    intern_to_cdn_conn,
    left_on='bodyId_post',
    right_on='bodyId_pre',
    suffixes=('_in', '_out')
).groupby(
    'bodyId_post_in'
).agg(
    {
        'weight_in' : 'sum',
        'weight_out' : 'sum'
    }
)

mutual_conn = mutual_conn[mutual_conn['weight_in'] > 90]

mutual_df = fetch_neurons(
    NC(bodyId = mutual_conn.index.values)
)[0]

mutual_df = mutual_df.set_index('bodyId')

mutual_conn['neurotransmitter'] = mutual_df.loc[mutual_conn.index.values]['predictedNt']
mutual_conn['entropy'] = mutual_df.loc[mutual_conn.index.values][['ntAcetylcholineProb', 'ntGabaProb', 'ntGlutamateProb']].apply(lambda x: -np.sum(x*np.log2(x)), axis=1)

plt.scatter(
    mutual_conn[mutual_conn['neurotransmitter'] == 'gaba']['weight_in'],
    mutual_conn[mutual_conn['neurotransmitter'] == 'gaba']['weight_out'],
    color = LAPIS,
    alpha = 1+mutual_conn[mutual_conn['neurotransmitter'] == 'gaba']['entropy'].values/np.log2(1/3),
    edgecolors=None,
    linewidths = 0,
)

plt.scatter(
    mutual_conn[mutual_conn['neurotransmitter'] == 'acetylcholine']['weight_in'],
    mutual_conn[mutual_conn['neurotransmitter'] == 'acetylcholine']['weight_out'],
    color = BYZANTINE,
    alpha = 1+mutual_conn[mutual_conn['neurotransmitter'] == 'acetylcholine']['entropy'].values/np.log2(1/3),
    edgecolors=None,
    linewidths = 0,
)

plt.scatter(
    mutual_conn[mutual_conn['neurotransmitter'] == 'glutamate']['weight_in'],
    mutual_conn[mutual_conn['neurotransmitter'] == 'glutamate']['weight_out'],
    color = SHAMROCK,
    alpha = 1+mutual_conn[mutual_conn['neurotransmitter'] == 'glutamate']['entropy'].values/np.log2(1/3),
    edgecolors=None,
    linewidths = 0,
)

plt.xlabel('Synapses received from CDNs')
plt.ylabel('Synapses onto CDNs')

plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().set_aspect('equal')
plt.gca().set_xlim(0,2200)
plt.gca().set_ylim(0,2200)
plt.gca().set_xticks([0,1000,2000])
plt.gca().set_yticks([0,1000,2000])
plt.gcf().set_size_inches(2,2)

plt.gcf().savefig('CDN_interneuron_mutual_synapses.svg')

In [ ]:
mutual_conn[mutual_conn['neurotransmitter'] == 'gaba'].iloc[:20]



In [ ]:
types = ['descending', 'interneuropil', 'interneuron']
fraction_of_inputs = {
    TYPE : 
        downstream_c2_grouped.loc[downstream_c2_grouped[downstream_c2_grouped['output_type'] == TYPE].index.values]['weight'].values/(
        fetch_neurons(NC(bodyId=downstream_c2_grouped[downstream_c2_grouped['output_type'] == TYPE].index.values))[0]['post'].values
        )
    for TYPE in types
}

In [ ]:
from scipy.stats import gaussian_kde

# gk = gaussian_kde(np.exp(fraction_of_inputs['descending']))

# plt.plot(
#     np.logspace(-4, 0, 100),
#     gk(np.exp(np.logspace(-4, 0, 100))),
#     color = SHAMROCK,
#     label = 'Descending'
# )

# plt.plot(
#     np.logspace(-4, 0, 100),
#     gaussian_kde(np.exp(fraction_of_inputs['interneuropil']))(np.exp(np.logspace(-4, 0, 100))),
#     color = BYZANTINE,
#     label = 'Interneuropil'
# )

plt.hist(
    fraction_of_inputs['descending'],
    bins = np.logspace(-4, 0, 20),
    color = LAPIS,
    #alpha = 0.3,
    stacked=True,
    label = 'Descending',
    histtype='step',
    linewidth = 3
)

plt.hist(
    fraction_of_inputs['interneuropil'],
    bins = np.logspace(-4, 0, 20),
    color = SHAMROCK,
    #alpha = 0.3,
    stacked=True,
    label = 'Interneuropil',
    histtype='step',
    linewidth = 3
)

plt.hist(
    fraction_of_inputs['interneuron'],
    bins = np.logspace(-4, 0, 20),
    color = '#58479E',
    #alpha = 0.3,
    stacked = True,
    label = 'Interneuron',
    histtype='step',
    linewidth = 3
)

plt.gca().set_ylabel('Number of neurons\nof class')
plt.gca().set_xlabel('Fraction of input synapses\ncoming from CDNs')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.gca().set_ylim(1, 1e2)
plt.gca().set_xscale('log')
plt.gca().set_yscale('log')

plt.gcf().set_size_inches(3,1.5)
plt.gcf().savefig('CDN_fraction_of_inputs_to_output_cells_by_class.svg')


In [ ]:
gk(np.logspace(-4, 0, 100))